# Pseudo Global Warming Climate Delta Calculation

### Read in necccessary libraries and packages

In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
import xarray as xr
import Ngl
import numpy as np
import glob

from dask_jobqueue import SLURMCluster
from dask.distributed import Client

import xesmf as xe

import metpy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

### Start up dask to run functions in parallel to speed things up

In [ ]:
cluster = SLURMCluster()
cluster.scale(2)
client = Client(cluster)

In [ ]:
client

### Read in the NAM data which will be used for regridding and vertical interpolation

In [ ]:
nam_including_coords_ds = xr.open_dataset('/data/meso/a/jtrapp/formax/namanl_218_20100510_1200_000.grb', engine='cfgrib',
                                           backend_kwargs={
                                                            'filter_by_keys': {'cfName': 'eastward_wind', 'typeOfLevel': 'isobaricInhPa'},
                                                            'errors': 'ignore'
                                                            }
                                         )
ds_out = nam_including_coords_ds.rename({'isobaricInhPa': 'plev', 'longitude': 'lon', 'latitude': 'lat'})

plev_out = ds_out.plev.values

---
### Setup a few functions to make things easier 

In [ ]:
def convert_vert_coords(ds, var, vert_levs, ds_ps=None):
    
    # This section will work with GFDL and MIROC since they have the same naming conventions
    try:
        
        # Flip the data to span from top to bottom instead of bottom to top (of atmosphere)
        data = np.flip(ds[var].values, axis=1) 
        
        time = ds.time.values
        
        # Read in the a and b coefficients used for converting from hybrid-sigma coordiantes to pressure levels
        a = ds['a'].values[::-1]
        b = ds['b'].values[::-1]
        
        # Use the pressure surface provided in the file
        ps = ds['ps'].values
        
        # Read in lats and lons
        lat = ds['lat'].values
        lon = ds['lon'].values
        
        # Convert from hybrid sigma coordinates to pressure coordinates using PyNGL
        val = Ngl.vinth2p(datai=data, hbcofa=a, hbcofb=b, plevo=vert_levs, psfc=ps,intyp=2, p0=1013.25, ii=1, kxtrp=False)
        
        # Output to an array
        new_ds = xr.DataArray(val, coords=[time, vert_levs, lat, lon], dims=['time', 'plev', 'lat', 'lon']).to_dataset(name=var)
        
    except:
        
        # Follows the same process as above, except uses separate pressure array
        data = ds[var].values
        time = ds.time.values
        a = ds['hyam'].values
        b = ds['hybm'].values
        ps = ds_ps['PS'].values
        lat = ds['lat'].values
        lon = ds['lon'].values
    
        val = Ngl.vinth2p(datai=data, hbcofa=a, hbcofb=b, plevo=vert_levs, psfc=ps, intyp=2, p0=1000, ii=1, kxtrp=False)
    
        new_ds = xr.DataArray(val, coords=[time, vert_levs, lat, lon], dims=['time', 'plev', 'lat', 'lon']).to_dataset(name=var)
    
    # Remove extremely large values and replace with nan
    return new_ds.where(new_ds[var] < 1000)

In [ ]:
def regrid(ds_in, ds_out, plev_out=None, vert_interp=True):
    
    # Regrids using NAM data read in from above
    # Set vert_interp to true when working with pressure level data, otherwise set to False for surface data
    # Use log-P to interpolate
    
    if vert_interp==True:
        # Reduce the vertical extent to 50-hPa instead of 10-hPa
        reduced_ds = ds_in.isel(plev=np.where(ds_in.plev.values >= 50)[0])
    
        # Change the vertical coordinate to ln(p)
        reduced_ds['plev'] = np.log(reduced_ds['plev'].values)
    
        # Change the vertical coordinate of the new grid to ln(p)
        plev_out = np.log(ds_out.plev.values)
    
        # Perform the interpolation
        climo_delta_interpolated = reduced_ds.interp({'plev': plev_out})
        
        # Change the pressure values back to the original values
        climo_delta_interpolated['plev'] = ds_out.plev.values
    
        # Change the name of the pressure levels to isobaric to match NAM
        climo_delta = climo_delta_interpolated.rename({'plev':'isobaric'})
        
    else:
        climo_delta = ds_in
    
    regridder = xe.Regridder(climo_delta, ds_out, 'bilinear', reuse_weights=True)
    
    return regridder(climo_delta)

In [ ]:
def interp_hourly(ds):
    
    # Interpolates the three hourly data, removing the 24 hour value since we only want up to 21
    ds_copy = ds.sel(hour=0)
    ds_copy['hour'] = 24
    return xr.concat([ds, ds_copy], dim='hour').interp({'hour':np.arange(0, 24, 3)})

In [ ]:
def process_decade_data(model, var, list_years, month):
    
    # Create a list for the different datasets
    ds_list = []
    
    # With NCAR data, pressure surface data needs to be read in seperately
    if model == 'NCAR':
        ds_ps_list = []
    
        # Create a dictionary for the variable names for NCAR
        ncar_var_names = {'ta':'T',
                          'ua':'U',
                          'va':'V',
                          'hus':'Q'}
        
        # Rename the NCAR variables to match those in the files
        var = ncar_var_names[var]
    
    # Read in the files using typical naming conventions
    files = sorted(glob.glob('/data/meso/a/mgrover4/'+model+'_6hour/'+var+'_*'))
    
    
    # If the file list is empty, try using another file naming convention (typically happens with NCAR data)
    if len(files) == 0:
        
        files = sorted(glob.glob('/data/meso/a/mgrover4/'+model+'_6hour/*.'+var+'.nc'))
    
    if model == 'MIROC':
        plevs = xr.open_dataset('/data/jtrapp/a/jtrapp/CMIP5/MIROC/ta_Amon_MIROC5_historical_r1i1p1_199001-199912.nc').plev.values/100
        
    elif model == 'GFDL':
        plevs = xr.open_dataset('/data/jtrapp/a/jtrapp/CMIP5/GFDL/ta_Amon_GFDL-CM3_historical_r1i1p1_199001-199412.nc').plev.values/100
        
    elif model == 'NCAR':
        plevs = xr.open_dataset('/data/jtrapp/a/jtrapp/CMIP5/NCAR/ta_Amon_CCSM4_historical_r1i1p1_195001-200512.nc').plev.values/100
        ps_files = sorted(glob.glob('/data/meso/a/mgrover4/'+model+'_6hour/*.PS.nc'))
        
    for year in list_years:
        
        # Convert the year into a string
        year = str(year)
        
        # Loop through the different files
        for i in range(len(files)):
            
            # For the MIROC data, 3D atmos variables are stored in monthly files, not yearly so different datetime is used
            if (model == 'MIROC') and ((var == 'ta') or (var == 'hus') or (var == 'ua') or (var == 'va')):
                datetime = year+month
            
            # For any data other than MIROC, use just the year
            else:
                datetime = year
            
            if datetime in files[i]:
                print(files[i])
                
                # Open dataset
                ds = xr.open_dataset(files[i]).sel(time = year+ '-' +month)
                
                # If NCAR dataset, read in the PS values
                if model == 'NCAR':
                    ds_ps = xr.open_dataset(ps_files[i]).sel(time = year+ '-' +month)
                    ds = convert_vert_coords(ds, var, plevs, ds_ps=ds_ps)
                
                else:
                    
                    # Convert from hybrid-sigma coordinates to pressure coordinates
                    ds = convert_vert_coords(ds, var, plevs)
            
                # Find hourly average from the full month of data
                ds = ds.groupby('time.hour').mean('time')
            
                # Linearly interpolate hourly data
                ds = interp_hourly(ds)
            
                # Add to the historical list of datasets
                ds_list.append(ds)
            
                print('Done with: '+year + ' ' + var)
    
    # Merge the years together
    ds = xr.concat(ds_list, dim='hour')
    
    # Finish with the horizontal regridding and log-p vertical interpolation
    ds = regrid(ds, ds_out, plev_out)
    
    # Average across the different hours
    return ds.groupby('hour').mean()

In [ ]:
def atmos_delta(var, model='MIROC', month='05', historical_years=list(range(1990, 2000)), future_years=list(range(2090, 2100))):
    
    # Calculate the delta using this function - make sure to use different month values for March (default is May)
    
    # Calulate historical average
    historical_ds = process_decade_data(model, var, historical_years, month)
    
    # Calculate future average
    future_ds = process_decade_data(model, var, future_years, month)
    
    return future_ds - historical_ds

--- 
## Calculate 3D Atmos Deltas

The default month in this section is the month of May, so if you would like to calculate another month, please input the month in the ```atmos_delta()``` funtion (ex. ```month='03'```) for March

### MIROC 5

In [ ]:
# Create a list of variables of 3D atmos deltas
atmos_vars = ['ta', 'ua', 'va', 'hus']

In [ ]:
# Create a list for the datasets
miroc_ds_list = []

# Loop through the different variables and calculate the PGW climate delta
for var in atmos_vars:
    miroc_ds_list.append(atmos_delta(var, 'MIROC').compute())

In [ ]:
# Merge into a single dataset
miroc_3d_atmos = xr.merge(miroc_ds_list)

In [ ]:
# Write to file
miroc_3d_atmos.to_netcdf('../preliminary_netcdf/MIROC_3D_vars_3hr.nc')

### GFDL-CM3

In [ ]:
# Create a list for the datasets
gfdl_ds_list = []

# Loop through the different variables and calculate the PGW climate delta
for var in atmos_vars:
    gfdl_ds_list.append(atmos_delta(var, 'GFDL'))

In [ ]:
# Merge the datasets
gfdl_3d_atmos = xr.merge(gfdl_ds_list)

# Write to file
gfdl_3d_atmos.to_netcdf('../preliminary_netcdf/GFDL_3D_vars_3hr.nc')

### NCAR-CCSM4

In [ ]:
# Create  list for the datasets
ncar_ds_list = []

# Loop through the different variables and calculate the PGW climate delta
for var in atmos_vars:
    ncar_ds_list.append(atmos_delta(var, 'NCAR').compute())

In [ ]:
# Merge the different variable datasets into one
ncar_3d_atmos = xr.merge(ncar_ds_list)

# Change names to match naming convention from MIROC and GFDL
ncar_3d_atmos.rename({'T':'ta',
                      'U':'ua',
                      'V':'va',
                      'Q':'hus'})

# Write to netcdf file in preliminary netcdf - used in the write to grib
ncar_3d_atmos.to_netcdf('../preliminary_netcdf/NCAR_3D_vars_3hr.nc')

---
## Surface Deltas

In [ ]:
def average_data(var, model, month=5, var_type='surface', plev_out=plev_out):
    
    # Pull in the files
    if 'NCAR' in model:
        
        files = sorted(glob.glob('/data/meso/a/mgrover4/'+model+'_6hour/*.'+var+'.nc'))
        
        if (var == 'T') or (var == 'Q'):
            
            ds_list = []
                       
            # Loop through the different time steps and use the lowest sigma level for temperature 
            for file in files:
                ds = xr.open_dataset(file, chunks={'time': 40}).isel(lev=-1)[var].to_dataset()
                #ds = ds.sel(time = ds['time.month'] == 5)
                ds_list.append(ds)
            
            # Merge all the time steps
            ds = xr.concat(ds_list, dim='time')
        
        else:
            
            ds_list = []
            
            for file in files:
                ds = xr.open_dataset(file, chunks={'time': 40})[var].to_dataset()
                ds_list.append(ds)
            ds = xr.concat(ds_list, dim='time')
    else:
        
        files = sorted(glob.glob('/data/meso/a/mgrover4/'+model+'_6hour/'+var+'_*'))
        ds = xr.open_mfdataset(files, chunks={'time': 40}, concat_dim='time', combine='nested')
    
    # Select may
    may_ds = ds.sel(time = (ds['time.month'] == month))
        
    # Subset historical years
    historical_ds = may_ds.sel(time = (may_ds['time.year'] < 2000))
    
    # Average historical - hourly
    historical_ds_mean_hourly = historical_ds.groupby('time.hour').mean('time').compute()
    print('processed hourly historical')
    
    # Subset future years
    future_ds = may_ds.sel(time = (may_ds['time.year'] > 2000))
    
    # Average future - hourly
    future_ds_mean_hourly = future_ds.groupby('time.hour').mean('time').compute()
    print('processed hourly future')
    
    
    # Find difference between future and past averages - hourly
    delta_ds_hourly = future_ds_mean_hourly[var] - historical_ds_mean_hourly[var]
    
    return delta_ds_hourly.to_dataset()

In [ ]:
def process_surface(model, variables, plot=False):
    
    datasets_hourly = []
    datasets_monthly = []
    for var in variables:
        ds_hourly = average_data(var, model)
        
        # Interpolate to 3-hourly if it is 6 hour data
        if 21 not in ds_hourly.hour.values:
            ds_hourly = interp_hourly(ds_hourly)

        datasets_hourly.append(ds_hourly)

        print('Processed '+var)
    
    # Merge hourly datasets
    delta_ds_hourly = xr.merge(datasets_hourly, compat='override')
    print(delta_ds_hourly)
    
    # Create Plots
    
    if plot:
        for hour in delta_ds_hourly.hour.values:
            if 'NCAR' in model:
                surface_plot(delta_ds_hourly, hour, model+' Surface')
            else:
                surface_plot(delta_ds_hourly, hour, model+' Surface')
    
    return delta_ds_hourly

In [ ]:
def interp_hourly(ds):
    ds_copy = ds.sel(hour=0)
    ds_copy['hour'] = 24
    return xr.concat([ds, ds_copy], dim='hour').interp({'hour':np.arange(0, 24, 3)})

---
## Calculate Surface Deltas

### MIROC 5

In [ ]:
miroc_hourly_ds = process_surface('MIROC', ['ps', 'psl', 'tas', 'uas', 'vas', 'huss'])

### GFDL-CM3

In [ ]:
gfdl_hourly_ds = process_surface('GFDL', ['ps', 'psl', 'tas', 'uas', 'vas', 'huss'])

### NCAR-CCSM4

In [ ]:
ncar_hourly_ds = process_surface('NCAR', ['T', 'Q', 'PS', 'PSL', 'TS', 'UBOT', 'VBOT'])

In [ ]:
# Rename NCAR variables to match naming convention from MIROC and GFDL
ncar_hourly_ds = ncar_hourly_ds.rename({'T':'tas',
                                        'Q':'huss',
                                        'PS':'ps',
                                        'PSL':'psl',
                                        'TS':'ts',
                                        'UBOT':'uas',
                                        'VBOT':'vas'})

### Write all the surface deltas to file

In [ ]:
regrid(miroc_hourly_ds, ds_out, vert_interp=False).to_netcdf('../preliminary_netcdf/MIROC_2d_vars_3hr.nc')

In [ ]:
regrid(gfdl_hourly_ds, ds_out, vert_interp=False).to_netcdf('../preliminary_netcdf/GFDL_2d_vars_3hr.nc')

In [ ]:
regrid(ncar_hourly_ds, ds_out, vert_interp=False).to_netcdf('../preliminary_netcdf/NCAR_2d_vars_3hr.nc')